In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
csv = pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
# tạo label khi positive là 1, negative là 0
label = csv['sentiment'].map({'positive':1, 'negative':0}).values
csv.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
# gán id cho từng từ 
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words = 5000) # chọn 5000 từ xuất hiện nhiều nhất
token.fit_on_texts(csv['review'].values)

Using TensorFlow backend.


In [0]:
token.word_index

In [0]:
csv['review'].values

array(["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to

In [0]:
X = token.texts_to_sequences(csv['review'].values)
y = label

In [0]:
X[0]

In [0]:
from keras.preprocessing.sequence import pad_sequences
# giới hạn chiều dài của câu 
X = pad_sequences(X, maxlen=300) # giới hạn 300 ngắn hơn sẽ pad thêm số 0, dài thì cắt

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.5, random_state = 42, stratify = y)

# phân tập train thành 2 tập train và validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42, stratify = y_train)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(20000, 300) (20000,) (5000, 300) (5000,)


In [0]:
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint

# thiết kế mạng 
input_layer = Input(shape = (300,))
emb_layer = Embedding(5000,100)(input_layer)
conv1d_layer = Conv1D(100, 3, activation='tanh')(emb_layer)
pool_layer = GlobalMaxPool1D()(conv1d_layer)

dense_layer = Dense(50, activation='sigmoid')(pool_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

model = Model(input = input_layer, output = output_layer)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics =['accuracy'])
print(model.summary())
save_model = ModelCheckpoint('check_point.h5',save_best_only= True, save_weights_only=  True)
model.fit(X_train,y_train,validation_data=(X_val,y_val), epochs= 20, batch_size=100, callbacks=[save_model])

In [0]:
model.load_weights("check_point.h5")
y_test_pred = model.predict(X_test)
y_test_pred = [1 if pred > 0.5 else 0 for pred in y_test_pred ]
import numpy as np
print("Acc:",(1 - np.sum(np.abs(y_test_pred- y_test))/len(y_test_pred)))

In [0]:
# TODO: Xay dung mo hin su dung LSTM de phan tich cảm xúc

In [0]:
# TODO: Xay dung mo hinh su dung ca 2 CNN va LSTM de phan tich cảm xúc